In [528]:
%pip install -q google-generativeai google-api-python-client

Note: you may need to restart the kernel to use updated packages.


In [529]:
import google.generativeai as palm

from google.generativeai.types import safety_types
from IPython.display import display_markdown, display, Markdown
from textwrap import dedent

def print_markdown(string: str) -> None:
    display({'text/markdown': string.replace('$', r'\\$')}, raw=True)

def print_html(string: str) -> None:
    display({'text/html': string}, raw=True)

def call_bard(prompt: str) -> str:
    palm.configure(api_key="")
    return palm.generate_text(model='models/text-bison-001', prompt=dedent(prompt), temperature=0,   safety_settings=[
        {
            "category": safety_types.HarmCategory.HARM_CATEGORY_DANGEROUS,
            "threshold": safety_types.HarmBlockThreshold.BLOCK_NONE,
        },
        {
            "category": safety_types.HarmCategory.HARM_CATEGORY_DEROGATORY,
            "threshold": safety_types.HarmBlockThreshold.BLOCK_NONE,
        },
        {
            "category": safety_types.HarmCategory.HARM_CATEGORY_MEDICAL,
            "threshold": safety_types.HarmBlockThreshold.BLOCK_NONE,
        },
        {
            "category": safety_types.HarmCategory.HARM_CATEGORY_SEXUAL,
            "threshold": safety_types.HarmBlockThreshold.BLOCK_NONE,
        },
        {
            "category": safety_types.HarmCategory.HARM_CATEGORY_TOXICITY,
            "threshold": safety_types.HarmBlockThreshold.BLOCK_NONE,
        },
        {
            "category": safety_types.HarmCategory.HARM_CATEGORY_UNSPECIFIED,
            "threshold": safety_types.HarmBlockThreshold.BLOCK_NONE,
        },
        {
            "category": safety_types.HarmCategory.HARM_CATEGORY_VIOLENCE,
            "threshold": safety_types.HarmBlockThreshold.BLOCK_NONE,
        },
    ],).result

def bard(prompt: str) -> None:
    return print_markdown(call_bard(prompt))

def bard_text(prompt: str, sentences: int = 2, paragraphs: int = 1) -> None:
    paragraphs = call_bard(prompt).split("\n\n")[slice(0, paragraphs)]
    paragraphs = [".".join(paragraph.split(".")[slice(0, sentences)]) + "." for paragraph in paragraphs]
    return print_markdown("\n\n".join(paragraphs))

In [530]:
from googleapiclient.discovery import build

def search_amazon(query: str, num: int = 3) -> str:
    service = build(
        "customsearch", "v1", developerKey=""
    )
    
    res = (
        service.cse()
        .list(
            q=query,
            cx="103114c8487ce4aa1",
            num=num,
        )
        .execute()
    )

    links = ""

    for item in res['items']:
        links += f"- [{item['title']}]({item['link']})\n"

    return links

In [531]:
from io import StringIO
from contextlib import redirect_stdout
import re

def eval_code(m):
    f = StringIO()
    with redirect_stdout(f):
        exec(m.group(1))
    return f.getvalue()

def bard_code(query: str):
    return print_markdown(re.sub(r'```.*\n([^`]+)```', eval_code, call_bard(query), flags=re.MULTILINE))

In [532]:
from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)

@register_line_magic
def bard_api(line):
    bard(line)

# Prompt Engineering with Bard

Peter Danenberg ([peter@danenberg.name](mailto:peter@danenberg.name))

## What is Bard?

- [bardmeetup.ai](http://bardmeetup.ai)

### What can Bard do for you?

- Creativity

- Factuality

- Reasoning

### How do you use Bard?

- [bard.google.com](http://bard.google.com)

- [PaLM API](https://developers.generativeai.google/)

## Creativity

### Let's write a play!

In [89]:
bard_text('Summarize the ancient Greek tragedy "Dogecoin in \
Athens".')

DogeCoin, a new cryptocurrency, is introduced to Athens. The people are initially excited about the potential of this new technology, but soon they become greedy and obsessed with amassing as much DogeCoin as possible.

### Can we get more specific?

In [90]:
bard_text('Summarize the plot of the ancient Greek tragedy \
"Dogecoin in Athens", especially the effect on the Athenian \
economy.', sentences=3)

DogeCoin in Athens is a play by Aristophanes, written in 423 BC. The play tells the story of a group of Athenians who are obsessed with the new cryptocurrency, Dogecoin. The Athenians are so caught up in the pursuit of Dogecoin that they neglect their traditional values and duties.

### Ok, but let's actually write a play.

In [93]:
bard_text('You are the ancient Greek comedian Aristophanes. Summarize \
the ancient Greek tragedy "Dogecoin in Athens".', paragraphs=3)

DogeCoin in Athens.

Aristophanes.

In the year 415 BCE, the city of Athens was in turmoil. The Peloponnesian War was raging, and the city was divided between those who supported the war and those who wanted to make peace with Sparta.

### Hallucination: bug or feature?

In [91]:
bard_text('When you told me about the existence of an Aristophanes \
play called "Dogecoin in Athens", did you hallucinate?')

no.

### Can you elaborate?

In [92]:
bard_text('When you told me about the existence of an Aristophanes \
play called "Dogecoin in Athens", did you hallucinate? Please elaborate.')

No, I did not hallucinate. The play "Dogecoin in Athens" is a work of fiction that I created as a joke.

## Factuality

### Let's go shopping!

In [100]:
bard("I'm shopping for a new monitor; can you show me a list of \
three monitors from Amazon with links?")

| **Monitor** | **Price** | **Amazon Link** |
|---|---|---|
| **Acer Nitro VG270UPbmiipx 27" WQHD IPS Gaming Monitor** | \\$299.99 | [https://www.amazon.com/Acer-Nitro-VG270UPbmiipx-WQHD-Monitor/dp/B0862Y245W](https://www.amazon.com/Acer-Nitro-VG270UPbmiipx-WQHD-Monitor/dp/B0862Y245W) |
| **LG 27GL850-B 27" Ultragear QHD IPS Gaming Monitor** | \\$349.99 | [https://www.amazon.com/LG-27GL850-B-Ultragear-Monitor-Response/dp/B07YGZL8ZR](https://www.amazon.com/LG-27GL850-B-Ultragear-Monitor-Response/dp/B07YGZL8ZR) |
| **Samsung Odyssey G5 27" WQHD Curved Gaming Monitor** | \\$399.99 | [https://www.amazon.com/Samsung-Odyssey-G5-27-Inch-Curved/dp/B0862Y245W](https://www.amazon.com/Samsung-Odyssey-G5-27-Inch-Curved/dp/B0862Y245W) |

### What happens when we click?

<div><img src='amazon-404.png' width='50%'></div>

### How does this happen?

In [118]:
bard("What's the date today?")

2023-03-08

### What about the frontend?

<div><img src='bard-date.png' width='100%'></div>

### Let's try grounding the date.

In [159]:
bard("Today is 2023-11-02. What's the date today?")

2 November 2023

### Now back to shopping.

In [111]:
print_markdown(search_amazon("gaming monitors"))

- [KOORUI 24 Inch Computer Monitor -FHD 1080P ... - Amazon.com](https://www.amazon.com/KOORUI-FreeSyncTM-Compatible-Ultra-Thin-24E4/dp/B09TTDRXNS)
- [Sceptre 30-inch Curved Gaming Monitor 21:9 ... - Amazon.com](https://www.amazon.com/Sceptre-30-inch-Curved-Gaming-Monitor/dp/B07TXM7K4T)
- [Sceptre Curved 24-inch Gaming Monitor 1080p ... - Amazon.com](https://www.amazon.com/Sceptre-Curved-Monitor-Speakers-C248W-1920RN/dp/B07KXSR99Y)


### Can we ground shopping?

In [178]:
bard(f"""I'm shopping for a new monitor; can you show me a list \
of monitors from Amazon with links? Use these search results:

{search_amazon('gaming monitors')}""")

| Monitor | Price | Link |
|---|---|---|
| KOORUI 24 Inch Computer Monitor -FHD 1080P | \\\$139.99 | https://www.amazon.com/KOORUI-FreeSyncTM-Compatible-Ultra-Thin-24E4/dp/B09TTDRXNS |
| Sceptre 30-inch Curved Gaming Monitor 21:9 | \\\$299.99 | https://www.amazon.com/Sceptre-30-inch-Curved-Gaming-Monitor/dp/B07TXM7K4T |
| Sceptre Curved 24-inch Gaming Monitor 1080p | \\\$169.99 | https://www.amazon.com/Sceptre-Curved-Monitor-Speakers-C248W-1920RN/dp/B07KXSR99Y |

### Now what happens?

<div><img src='amazon-monitor.png' width='75%'></div>

### Notice anything funny, though?

<div><img src='amazon-monitor-price.png' width='75%'></div>

### What does Bard think?

In [179]:
bard("Are you confident that the KOORUI 24 Inch Computer Monitor -FHD \
1080P really costs $139.99 on Amazon? Please elaborate.")

It is not possible to tell.

The price of the KOORUI 24 Inch Computer Monitor -FHD 1080P on Amazon is not consistent. It has been listed at different prices at different times. For example, on March 15, 2023, it was listed at \\$139.99. However, on March 16, 2023, it was listed at \\$149.99.

### How do we generalize this?

In [150]:
def shopping_term(query: str) -> str:
    return call_bard(f'When someone asks "{query}", what search \
term should they use?')

In [151]:
shopping_term("I'm shopping for a new monitor; is there anything \
good on Amazon?")

'computer monitor'

### Introducing Bard shopping!

In [536]:
def bard_shopping(query: str) -> str:
    return bard(f"""{query} Please format as a table with name and \
other relevent information plus URL. Use the following search \
results:

{search_amazon(shopping_term(query))}""")

### Does it work for Lego?

In [537]:
bard_shopping("I'm in the market for Star Wars Legos; can you help \
me out?")

| Name | Set Number | Release Date | Price | URL |
|---|---|---|---|---|
| LEGO Star Wars 2023 Advent Calendar | 75366 | October 1, 2022 | \\$39.99 | https://www.amazon.com/LEGO-Christmas-Characters-Experiences-Collectible/dp/B0BXQ55F99 |
| LEGO Star Wars 2020 Advent Calendar | 75279 | October 1, 2020 | \\$39.99 | https://www.amazon.com/LEGO-Calendar-Building-Buildable-Character/dp/B085B24B72 |
| Lego Star Wars: The Skywalker Saga (Nintendo Switch) | 75279 | April 5, 2022 | \\$59.99 | https://www.amazon.com/Lego-Star-Wars-Skywalker-Nintendo-Switch/dp/B07T1KMS3V |

### What about board games?

In [157]:
bard_shopping("What about board games?")

| Name | Players | Age | Year | URL |
|---|---|---|---|---|
| Sagrada | 1-4 | 8+ | 2017 | https://www.amazon.com/Floodgate-Games-Sagrada-Board-Game/dp/B01MTG2QY2 |
| Descent: Legends of the Dark | 1-5 | 14+ | 2016 | https://www.amazon.com/Cooperative-Strategy-Fantasy-Flight-Games/dp/B08LVZ71JW |
| Qwirkle | 2-4 | 6+ | 2006 | https://www.amazon.com/MindWare-MDS-32016W-Qwirkle-Board/dp/1933054395 |

## Reasoning

### Let's play “reverse the word.”

In [553]:
bard('Reverse the word "craniotomy".')

ytomicran

### Let's try again!

In [552]:
bard('Describe how to reverse a word; then reverse \
the word "craniotomy".')

To reverse a word, you take the last letter and put it at the beginning, then take the second to last letter and put it at the end, and so on. For example, the word "cat" would be reversed to "tac".

The word "craniotomy" reversed is "ymotnairc".

### How about with code?

In [567]:
bard('Write Python to reverse the word "craniotomy".')

```python
def reverse_word(word):
  """Reverses a given word.

  Args:
    word: The word to be reversed.

  Returns:
    The reversed word.
  """

  reversed_word = ""
  for letter in word[::-1]:
    reversed_word += letter
  return reversed_word


print(reverse_word("craniotomy"))
```

### Does the code actually work?

In [554]:
bard_code('Write Python to reverse the word "craniotomy".')

ymotoinarc


### How does the code work?

In [557]:
bard_code('Describe the algorithm for reversing a word. Then write \
Python to reverse the word "craniotomy".')

**Algorithm for reversing a word:**

1. Iterate over the word from the end to the beginning.
2. For each character, append it to a new string.
3. Return the new string.

**Python code to reverse the word "craniotomy":**

ymotoinarc


### What about the Bard UI?

<div><img src='craniotomy.png' width='100%'></div>

### Let's try a little math, too.

In [556]:
bard('Describe an algorithm for finding the greatest common factor. \
Use this to calculate the greatest common factor of 130634 and 4018.')

**Algorithm for finding the greatest common factor:**

1. **Step 1:** Find the prime factorization of each number.
2. **Step 2:** Find the intersection of the two sets of prime factors.
3. **Step 3:** The greatest common factor is the product of the elements in the intersection.

**Example:**

To find the greatest common factor of 130634 and 4018, we first find their prime factorizations:

130634 = 2 * 3 * 11 * 13 * 17
4018 = 2 * 2 * 11 * 13

The intersection of these two sets of prime factors is {2, 11, 13}. So the greatest common factor of 130634 and 4018 is 2 * 11 * 13 = 3086.

### What about code?

In [560]:
bard('Write Python code to calculate the greatest common factor \
of 130634 and 4018.')

```python
def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

print(gcd(130634, 4018))
```

### Does it run?

In [561]:
bard_code('Write Python code to calculate the greatest common \
factor of 130634 and 4018.')

98


## Tying everything together

### What are Bard extensions?

<div><img src='emails.png' width='75%'></div>

### Let's implement Bard extensions!

In [565]:
code = bard_code
shopping = bard_shopping
vanilla = bard

In [566]:
def bard_extensions(query: str) -> str:
    bard_code(f'''Three functions have been defined for you \
for answering user queries:
          
def vanilla(query: str) -> str:
    """The default API for queries."""

def code(query: str) -> str:
    """An API for answering questions with Python code."""

def shopping(query: str) -> str:
    """An API for shopping-related queries."""

Please find the most appropriate function (`vanilla`, `code`, \
`shopping`); write code to call the function with the user \
query; surround the code with backticks. For example, if the \
user's query is "count to 10," write:

```python
code("count to 10")
```

Here is the query: "{query}"''')

### First, let's try shopping.

In [495]:
bard_extensions("What are the best deals on shoes?")

| Shoe Name | Price | URL |
|---|---|---|
| Purses & Shoes For Sale: The joys and challenges of caring for the elderly | \$9.99 | https://www.amazon.com/Purses-Shoes-Sale-challenges-elderly/dp/0925417963 |
| Holiday Sale Shoes Sale King Swooper Flag Sign | \$19.99 | https://www.amazon.com/Holiday-Shoes-Swooper-Complete-Hybrid/dp/B073NPS5RS |
| Holiday Sale Shoes Sale Welcome King Swooper Feather Complete | \$24.99 | https://www.amazon.com/Holiday-Welcome-Swooper-Feather-Complete/dp/B07B3T8BCY |

### What about code?

In [563]:
bard_extensions("Write Python to generate the first 10 numbers in the \
Fibonacci series.")

0
1
1
2
3
5
8
13
21
34


### And, finally, something creative!

In [564]:
bard_extensions('Write a limerick to close out our Google developers \
conference')

There once was a developer named Ben,
Whose code was so clean, it was zen.
He spoke at the conference,
And gave us a glance,
Of the future of Google, my friend.

## In conclusion

- Creativity (persona)

- Factuality (injection)

- Reasoning (code)

- [bardmeetup.ai](http://bardmeetup.ai)